In [ ]:
# ratio_i = min(weight_i, max_payload) / (distance_i * 2.)

In [ ]:
import random
import time
from operator import attrgetter

In [ ]:
# load input data #############################
file = open(r'/kaggle/input/hashcode-drone-delivery/busy_day.in', 'r')

In [ ]:
###############################################
lines = file.readlines()
lines = [line.rstrip('\n').split(' ') for line in lines]

int_list = []
for line in lines:
    int_list.extend([[int(ele) for ele in line]])


###############################################
number_of_drones = int_list[0][2]
drones_location_at_start_in_warehouse1 = int_list[4]
max_payload = int_list[0][4]
turns = int_list[0][3]


# Create classes #############################

class Drone():
    def __init__(self, x, y, name):
        self.x = x
        self.y = y
        self.name = name
        self.weight_max = max_payload    # 5.
        self.order_in_progress_1 = []
        self.order_in_progress_2 = []
        self.cargo_list_1 = []
        self.cargo_list_2 = []
        self.weight = 0.
        self.x_target = []
        self.x_target_1 = []
        self.x_target_2 = []
        self.y_target = []
        self.y_target_1 = []
        self.y_target_2 = []
        self.speed_x = 0.
        self.speed_y = 0.
        self.turns_waited = 0
        self.status = 'reorienting'  # convention: 'reorienting', 'sending', 'unloading', 'returning', 'at hub'
# D1 = Drone(x=1, y=3, name='D1')


class Product():
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight
# P1 = Product(name='P1', weight=1.)


class Order():
    def __init__(self, name, x, y, products):
        self.name = name
        self.x = x
        self.y = y
        self.products = products
        self.weight = sum([self.products[i].weight for i in range(0, len(self.products))])
        self.status = 'open'   # convention: 'open', 'in progress', 'closed'
# O1 = Order(name='O1', x=3, y=6, products=[P1, P1, P2])


class Hub():
    def __init__(self, x, y, name):
        self.x = x
        self.y = y
        self.name = name
        self.products = []
        self.weight = 0.
        self.status = 'full'    # convention: 'full', 'empty'
# H1 = Hub(x=1, y=1, name='H1')
# H1.products = [P1, P1, P2, P3, P5]
# H1.weight = sum([product.weight for product in H1.products])


########################################################################################
list_of_drones = []
for i in range(number_of_drones):
    list_of_drones.append(Drone(x=drones_location_at_start_in_warehouse1[0],
                                y=drones_location_at_start_in_warehouse1[1],
                                name='D'+str(i)))


###########
number_of_product_types = int_list[1][0]
products_types_weights = int_list[2]

list_of_products = []
for i in range(number_of_product_types):
    list_of_products.append(Product(name='P'+str(i), weight=products_types_weights[i]))


###########
number_of_warehouses = int_list[3][0]

list_of_hubs = []
for i in range(number_of_warehouses):
    warehouse_i_location = int_list[4 + i * 2]
    warehouse_i_items = int_list[5 + i * 2]
    list_of_hubs.append(Hub(x=warehouse_i_location[0], y=warehouse_i_location[1], name='H'+str(i)))

    for x in range(number_of_product_types):
        list_of_hubs[i].products.extend([list_of_products[x]] * warehouse_i_items[x])

    list_of_hubs[i].weight = sum([product.weight for product in list_of_hubs[i].products])


###########
number_of_orders = int_list[3 + number_of_warehouses*2 + 1][0]

list_of_orders = []

for i in range(number_of_orders):
    order_i_location = int_list[3 + number_of_warehouses*2 + 2 + i*3]
    order_i_items_list = int_list[3 + number_of_warehouses*2 + 4 + i*3]
    list_of_orders.append(Order(name='O'+str(i),
                                x=order_i_location[0],
                                y=order_i_location[1],
                                products=[]))
    for item in order_i_items_list:
        list_of_orders[i].products.append(list_of_products[item])

    list_of_orders[i].weight = sum([product.weight for product in list_of_orders[i].products])

    list_of_orders[i].products.sort(key = attrgetter('weight'), reverse = True)   # True = Ascending
    # list_of_orders[i].products.sort(key = attrgetter('weight'), reverse = False)   # True = Ascending


#### General formulas ################################################################################
def calculate_distance(object1, object2):
    distance = ( (object1.x - object2.x)**2. + (object1.y - object2.y)**2. ) **(1./2.)
    return distance


def find_nearest(object_itself, list_of_locations):
    distance_to_locations = []
    for location in list_of_locations:
        distance_to_locations.append(calculate_distance(object_itself, location))
    nearest_location = list_of_locations[distance_to_locations.index(min(distance_to_locations))]
    return nearest_location


def find_heaviest_hub(list_of_hubs):
    heaviest_hub = list_of_hubs[[hub.weight for hub in list_of_hubs].index(max([hub.weight for hub in list_of_hubs]))]
    return heaviest_hub


def find_best_order(hub, list_of_open_orders):

    ratio= -999999.99
    best_order = []
    best_order_weight = []

    for order in list_of_open_orders:
        temp_order_products_list = [product for product in order.products]
        temp_hub_products_list = [product for product in hub.products]
        is_in_stock_list = []

        while len(temp_order_products_list) > 0:
            intersection = list(set(temp_order_products_list).intersection(set(temp_hub_products_list)))
            if len(intersection) > 0:
                is_in_stock_list.append(intersection[0])
                temp_order_products_list.remove(intersection[0])
                temp_hub_products_list.remove(intersection[0])
            else:
                break

        if len(is_in_stock_list)>0:
            weight_i = sum([product.weight for product in is_in_stock_list])
            distance_i = calculate_distance(hub, order)

            ratio_i = min(weight_i, max_payload) / (distance_i * 2.)
            #ratio_i = (weight_i / order.weight) / (distance_i * 2.) + (1.0 if weight_i<max_payload else 0.0)
            #ratio_i = weight_i / (distance_i * 2.)

            if ratio_i >= ratio:
                best_order = order
                best_order_weight = weight_i
                ratio = ratio_i

    return best_order, best_order_weight


def find_random_other_location(object_itself, list_of_locations):
    temp_list_of_locations = []
    temp_list_of_locations.extend(list_of_locations)
    # temp_list_of_locations.remove(object_itself)
    return temp_list_of_locations[random.randint(0, len(temp_list_of_locations)-1)]


###########################################################
submission_commands = []
number_of_total_commands = 0
number_of_simulation_steps = 0
# Example command: 0 L 1 2 3 (Command to drone 0, load at warehouse 1 products of product type 2, three of them.
# Example command: 0 D 1 2 3 (Command to drone 0, deliver for order 1 items of product type 2, three of them.
# Example command: 0 W 3 (Command to drone 0, wait for three turns)

# submission_commands.append('0 L 1 2 3\n')
# number_of_total_commands +=1
kpi_free_capacity = []


#### Iteration #############################################
def simulation_step():
    global number_of_total_commands
    global number_of_simulation_steps
    number_of_simulation_steps +=1

    for drone in list_of_drones:

            nearest_hub = find_nearest(object_itself=drone, list_of_locations=list_of_hubs)
            temp_list_of_full_hubs = [hub for hub in list_of_hubs if (hub.weight > 0)]
            temp_list_of_full_hubs_excluding_nearest_hub = [hub for hub in list_of_hubs if (hub.weight > 0 and hub != nearest_hub)]
            temp_list_of_open_orders = [order for order in list_of_orders if (order.status == 'open')]

            if drone.status == 'reorienting':

                if number_of_simulation_steps>1 and number_of_simulation_steps<10:
                    nearest_full_hub = find_random_other_location(object_itself=drone, list_of_locations=temp_list_of_full_hubs)
                    next_order_check_1, next_order_check_1_weight = find_best_order(nearest_full_hub, temp_list_of_open_orders)
                else:
                    nearest_full_hub = find_nearest(object_itself=drone, list_of_locations=temp_list_of_full_hubs)
                    next_order_check_1, next_order_check_1_weight = find_best_order(nearest_full_hub, temp_list_of_open_orders)

                ## Debug
                # test_temp_list_of_full_hubs = [hub for hub in list_of_hubs if (hub.weight > 0)]
                # test_temp_list_of_open_orders = [order for order in list_of_orders if (order.status == 'open')]
                # test_nearest_full_hub = find_nearest(object_itself=list_of_drones[0], list_of_locations=test_temp_list_of_full_hubs)
                # test_next_order_check_1 = find_best_order(test_nearest_full_hub, test_temp_list_of_open_orders)
                # test_next_order_check_1.weight
                # test_next_order_check_1.products
                # test_next_order_check_1.products[0].weight
                # test_next_order_check_1.products[0].name
                # [product.name for product in test_nearest_full_hub.products]
                # test_next_order_check_1.products in test_nearest_full_hub.products
                #
                # order_products = [product for product in test_next_order_check_1.products]
                # order_products = test_next_order_check_1.products
                #
                # list(set(order_products).intersection(set(hub_products)))
                # list(set([order_products]).intersection(set(hub_products)))


                while True:
                    if next_order_check_1_weight != []:
                        drone.x_target = nearest_full_hub.x
                        drone.y_target = nearest_full_hub.y
                        drone.status = 'returning'
                        #print('reorienting ->', drone.status, 'to nearest_full_hub', nearest_full_hub.name)
                        break

                    else:
                        other_hub_1 = find_nearest(object_itself=drone, list_of_locations=temp_list_of_full_hubs_excluding_nearest_hub)
                        next_order_check_2, next_order_check_2_weight = find_best_order(other_hub_1, temp_list_of_open_orders)
                        if next_order_check_2_weight != []:
                            drone.x_target = other_hub_1.x
                            drone.y_target = other_hub_1.y

                            drone.status = 'returning'
                            #print('reorienting ->', drone.status, 'to 2nd nearest full hub', other_hub_1.name)
                            break

                        else:
                            other_hub_2 = find_heaviest_hub(temp_list_of_full_hubs_excluding_nearest_hub)
                            next_order_check_3, next_order_check_3_weight = find_best_order(other_hub_2, temp_list_of_open_orders)
                            if next_order_check_3_weight != []:
                                drone.x_target = other_hub_2.x
                                drone.y_target = other_hub_2.y

                                drone.status = 'returning'
                                #print('reorienting ->', drone.status, 'to heaviest other_hub_2', other_hub_2.name)
                                break

                            else:
                                other_hub_3 = find_random_other_location(object_itself=nearest_hub, list_of_locations=temp_list_of_full_hubs_excluding_nearest_hub)
                                next_order_check_4, next_order_check_4_weight = find_best_order(other_hub_3, temp_list_of_open_orders)
                                if next_order_check_4_weight != []:
                                    drone.x_target = other_hub_3.x
                                    drone.y_target = other_hub_3.y

                                    drone.status = 'returning'
                                    #print('reorienting ->', drone.status, 'to random other_hub_3', other_hub_3.name)
                                    break

                                else:

                                    #print('No best orders found ---> waiting')
                                    submission_commands.append(str(int(drone.name.lstrip('D')))
                                                               + ' ' + 'W'
                                                               + ' ' + '1'
                                                               + '\n')

                                    break


            elif drone.status == 'at hub':
                # then load

                if len(temp_list_of_open_orders)>0:
                    # order_in_progress_1 = find_nearest(object_itself=drone, list_of_locations=temp_list_of_open_orders)
                    order_in_progress_1 = find_best_order(hub=nearest_hub, list_of_open_orders=temp_list_of_open_orders)[0]

                    if order_in_progress_1==[]:
                        drone.status = 'reorienting'
                        #print('at hub', nearest_hub.name, ' ->', drone.status, ' order 1 lists empty')
                        break

                    else:
                        temp_order_products_list_1 = [product for product in order_in_progress_1.products]


                        if len(temp_list_of_open_orders)==1:
                            order_in_progress_2 = []
                            temp_order_products_list_2 = []
                        else:
                            order_in_progress_2 = find_nearest(object_itself=order_in_progress_1, list_of_locations=[order for order in list_of_orders if (order.status == 'open' and order!=order_in_progress_1)])
                            temp_order_products_list_2 = [product for product in order_in_progress_2.products]

                        temp_hub_products_list_1 = [product for product in nearest_hub.products]

                        is_in_stock_list_1 = []
                        is_in_stock_list_2 = []

                        while len(temp_order_products_list_1)>0:
                            intersection_1 = list(set(temp_order_products_list_1).intersection(set(temp_hub_products_list_1)))
                            if len(intersection_1)>0:
                                is_in_stock_list_1.append(intersection_1[0])
                                temp_order_products_list_1.remove(intersection_1[0])
                                temp_hub_products_list_1.remove(intersection_1[0])
                            else:
                                break

                        if len(is_in_stock_list_1)>0:

                            drone.x_target = order_in_progress_1.x
                            drone.y_target = order_in_progress_1.y
                            drone.status = 'sending'
                            #print('at hub->', drone.status, 'x_target:', drone.x_target, 'y_target:', drone.y_target)

                            order_in_progress_1.status = 'in progress'

                            drone.order_in_progress_1 = order_in_progress_1

                            while (drone.weight_max - drone.weight) > is_in_stock_list_1[0].weight:

                                submission_commands.append(str(int(drone.name.lstrip('D')))
                                                           + ' ' + 'L'
                                                           + ' ' + str(int(nearest_hub.name.lstrip('H')))
                                                           + ' ' + str(int(is_in_stock_list_1[0].name.lstrip('P')))
                                                           + ' ' + '1'
                                                           + '\n')

                                number_of_total_commands += 1

                                drone.cargo_list_1.append(is_in_stock_list_1[0])

                                drone.weight = sum([product.weight for product in drone.cargo_list_1]) + sum([product.weight for product in drone.cargo_list_2])

                                nearest_hub.products.remove(is_in_stock_list_1[0])
                                is_in_stock_list_1.remove(is_in_stock_list_1[0])
                                if len(is_in_stock_list_1)==0:
                                    break

                            if len(is_in_stock_list_1)>0:
                                while (drone.weight_max - drone.weight) > is_in_stock_list_1[-1].weight:

                                    submission_commands.append(str(int(drone.name.lstrip('D')))
                                                               + ' ' + 'L'
                                                               + ' ' + str(int(nearest_hub.name.lstrip('H')))
                                                               + ' ' + str(int(is_in_stock_list_1[-1].name.lstrip('P')))
                                                               + ' ' + '1'
                                                               + '\n')

                                    number_of_total_commands += 1

                                    drone.cargo_list_1.append(is_in_stock_list_1[-1])

                                    drone.weight = sum([product.weight for product in drone.cargo_list_1]) + sum([product.weight for product in drone.cargo_list_2])

                                    nearest_hub.products.remove(is_in_stock_list_1[-1])
                                    is_in_stock_list_1.remove(is_in_stock_list_1[-1])
                                    if len(is_in_stock_list_1)==0:
                                        break


                        if (drone.weight_max - drone.weight) > (drone.weight_max * 0.001):

                            temp_hub_products_list_2 = [product for product in nearest_hub.products]     # refresh

                            while len(temp_order_products_list_2) > 0:
                                intersection_2 = list(set(temp_order_products_list_2).intersection(set(temp_hub_products_list_2)))
                                if len(intersection_2) > 0:
                                    is_in_stock_list_2.append(intersection_2[0])
                                    temp_order_products_list_2.remove(intersection_2[0])
                                    temp_hub_products_list_2.remove(intersection_2[0])
                                else:
                                    break

                            if len(is_in_stock_list_2) > 0 and (((drone.weight_max - drone.weight) > is_in_stock_list_2[0].weight) or (((drone.weight_max - drone.weight) > is_in_stock_list_2[-1].weight))):
                                order_in_progress_2.status = 'in progress'
                                drone.x_target_2 = order_in_progress_2.x
                                drone.y_target_2 = order_in_progress_2.y

                                drone.order_in_progress_2 = order_in_progress_2

                                while (drone.weight_max - drone.weight) > is_in_stock_list_2[0].weight:

                                    submission_commands.append(str(int(drone.name.lstrip('D')))
                                                               + ' ' + 'L'
                                                               + ' ' + str(int(nearest_hub.name.lstrip('H')))
                                                               + ' ' + str(int(is_in_stock_list_2[0].name.lstrip('P')))
                                                               + ' ' + '1'
                                                               + '\n')

                                    number_of_total_commands += 1

                                    drone.cargo_list_2.append(is_in_stock_list_2[0])

                                    drone.weight = sum([product.weight for product in drone.cargo_list_1]) + sum([product.weight for product in drone.cargo_list_2])

                                    nearest_hub.products.remove(is_in_stock_list_2[0])
                                    is_in_stock_list_2.remove(is_in_stock_list_2[0])
                                    if len(is_in_stock_list_2) == 0:
                                        break

                                if len(is_in_stock_list_2)>0:
                                    while (drone.weight_max - drone.weight) > is_in_stock_list_2[-1].weight:

                                        submission_commands.append(str(int(drone.name.lstrip('D')))
                                                                   + ' ' + 'L'
                                                                   + ' ' + str(int(nearest_hub.name.lstrip('H')))
                                                                   + ' ' + str(int(is_in_stock_list_2[-1].name.lstrip('P')))
                                                                   + ' ' + '1'
                                                                   + '\n')

                                        number_of_total_commands += 1

                                        drone.cargo_list_2.append(is_in_stock_list_2[-1])

                                        drone.weight = sum([product.weight for product in drone.cargo_list_1]) + sum([product.weight for product in drone.cargo_list_2])

                                        nearest_hub.products.remove(is_in_stock_list_2[-1])
                                        is_in_stock_list_2.remove(is_in_stock_list_2[-1])
                                        if len(is_in_stock_list_2) == 0:
                                            break
                        else:
                            drone.order_in_progress_2=[]


                        kpi_free_capacity.append(((drone.weight_max - drone.weight) / drone.weight_max) * 100.)

                        nearest_hub.weight = sum([product.weight for product in nearest_hub.products])


                        if len(drone.cargo_list_1)==0 and len(drone.cargo_list_2)==0:
                            # travel to other hub, if problems with orders at this hub
                            drone.status = 'reorienting'

                            # drone.status = 'returning'
                            # other_hub = find_random_other_location(object_itself=nearest_hub, list_of_locations=temp_list_of_full_hubs_excluding_nearest_hub)
                            # drone.x_target = other_hub.x
                            # drone.y_target = other_hub.y

                            drone.order_in_progress_1 = []
                            drone.order_in_progress_2 = []

                            #print('at hub', nearest_hub.name, ' ->', drone.status, ' both cargo lists empty')

                else:
                    #print('currently no open orders at hub', nearest_hub.name, ' ---> waiting')
                    submission_commands.append(str(int(drone.name.lstrip('D')))
                                               + ' ' + 'W'
                                               + ' ' + '1'
                                               + '\n')


            elif drone.status == 'sending':
                # continue to deliver
                drone.speed_x = 1 if drone.x_target > drone.x else -1 if drone.x_target < drone.x else 0
                drone.speed_y = 1 if drone.y_target > drone.y else -1 if drone.y_target < drone.y else 0
                drone.x = drone.x + drone.speed_x
                drone.y = drone.y + drone.speed_y
                drone.status = 'unloading' if (drone.x == drone.x_target and drone.y == drone.y_target) else 'sending'
                #print('sending->', drone.status, 'x:', drone.x, drone.x_target, 'y:', drone.y, drone.y_target)


            elif drone.status == 'returning':
                # continue to return
                drone.speed_x = 1 if drone.x_target > drone.x else -1 if drone.x_target < drone.x else 0
                drone.speed_y = 1 if drone.y_target > drone.y else -1 if drone.y_target < drone.y else 0
                drone.x = drone.x + drone.speed_x
                drone.y = drone.y + drone.speed_y
                drone.status = 'at hub' if (drone.x == drone.x_target and drone.y == drone.y_target) else 'returning'
                #print('returning ->', drone.status, 'x:', drone.x, drone.x_target, 'y:', drone.y, drone.y_target)


            elif drone.status == 'unloading':
                # unload

                if drone.x==drone.x_target and drone.y==drone.y_target and len(drone.cargo_list_1)>0:

                    while len(drone.cargo_list_1) > 0:

                        submission_commands.append(str(int(drone.name.lstrip('D')))
                                                   + ' ' + 'D'
                                                   + ' ' + str(int(drone.order_in_progress_1.name.lstrip('O')))
                                                   + ' ' + str(int(drone.cargo_list_1[0].name.lstrip('P')))
                                                   + ' ' + '1'
                                                   +'\n')

                        number_of_total_commands +=1

                        drone.order_in_progress_1.products.remove(drone.cargo_list_1[0])
                        drone.cargo_list_1.remove(drone.cargo_list_1[0])


                    drone.weight = sum([product.weight for product in drone.cargo_list_1]) + sum([product.weight for product in drone.cargo_list_2])

                    if len(drone.order_in_progress_1.products)==0:
                        drone.order_in_progress_1.status = 'closed'
                        list_of_orders.remove(drone.order_in_progress_1)
                    else:
                        drone.order_in_progress_1.status = 'open'
                        drone.order_in_progress_1.weight = sum([product.weight for product in drone.order_in_progress_1.products])

                    drone.order_in_progress_1 = []


                    if len(drone.cargo_list_2)>0:
                        drone.x_target = drone.x_target_2
                        drone.y_target = drone.y_target_2
                        drone.status = 'sending'
                        #print('unloading->', drone.status, '2of2')

                    else:
                        drone.status = 'reorienting'
                        #print('unloading->', drone.status, '1of1')
                        drone.order_in_progress_2 = []
                        drone.x_target = []
                        drone.y_target = []
                        drone.y_target_2 = []
                        drone.y_target_2 = []


                elif drone.x==drone.x_target_2 and drone.y==drone.y_target_2 and len(drone.cargo_list_1)==0 and len(drone.cargo_list_2)>0:
                    while len(drone.cargo_list_2) > 0:
                        submission_commands.append(str(int(drone.name.lstrip('D')))
                                                   + ' ' + 'D'
                                                   + ' ' + str(int(drone.order_in_progress_2.name.lstrip('O')))
                                                   + ' ' + str(int(drone.cargo_list_2[0].name.lstrip('P')))
                                                   + ' ' + '1'
                                                   + '\n')

                        number_of_total_commands += 1

                        drone.order_in_progress_2.products.remove(drone.cargo_list_2[0])
                        drone.cargo_list_2.remove(drone.cargo_list_2[0])


                    drone.weight = sum([product.weight for product in drone.cargo_list_1]) + sum([product.weight for product in drone.cargo_list_2])

                    if len(drone.order_in_progress_2.products) == 0:
                        drone.order_in_progress_2.status = 'closed'
                        list_of_orders.remove(drone.order_in_progress_2)
                    else:
                        drone.order_in_progress_2.status = 'open'
                        drone.order_in_progress_2.weight = sum([product.weight for product in drone.order_in_progress_2.products])

                    drone.order_in_progress_2 = []

                    drone.status = 'reorienting'
                    #print('unloading->', drone.status, '2of2')
                    drone.x_target = []
                    drone.y_target = []
                    drone.y_target_2 = []
                    drone.y_target_2 = []


                else:
                    print('do some debugging')

            else:
                print('unknown case')

# simulation_step()


#### Status Report #########################################
def status_report():
    print("-----------------------------------------")

    for drone in list_of_drones:
        print("Drone", drone.name, " x=", drone.x," -> ", drone.x_target, "  y=", drone.y, " -> ", drone.y_target,
              "  weight=", drone.weight, " status=", drone.status, "\n",
              # [product.name for product in drone.cargo_list_1],
              sep="")
    print("")

    for order in list_of_orders:
        print("Order", order.name, " x=", order.x, " y=", order.y,
              " weight=", order.weight, " status=", order.status, "\n",
              # [product.name for product in order.products],
              sep="")
    print("")

    for hub in list_of_hubs:
        print("Hub", hub.name, " x=", hub.x, " y=", hub.y, " weight=", hub.weight, "\n",
              # [product.name for product in hub.products],
              sep="")
    print("-----------------------------------------")


# status_report()

# simulation_step()
# status_report()
# simulation_step()
# status_report()
# simulation_step()
# status_report()
# simulation_step()
# status_report()
# simulation_step()
# status_report()



#### Run simulation ###################################################
start_time = time.time()

step=1
# while len(list_of_orders)>0:
# while len(list_of_orders)>1100:
# while step<turns:
# while step<10:
# while step<100:
# while step<1000:
# while step<5000:
# while step<10000:
# while step<20000:
# while step<50000:
while (len(list_of_orders)>0 and step<turns and number_of_total_commands<turns):
    print("############################ iteration:", step, "############################")
    simulation_step()
    # status_report()
    print('#####')
    print('##### Number of iterations:', number_of_simulation_steps)
    print('##### Number of orders closed:', number_of_orders - len(list_of_orders))
    print('##### Number of commands:', number_of_total_commands)
    step = step + 1


stop_time = time.time()
simulation_time_in_minutes = round((stop_time - start_time) / 60., 2)
print('##### Minutes:', simulation_time_in_minutes)
print('##### Free Capacity on average:', round(sum(kpi_free_capacity) / float(len(kpi_free_capacity)), 2))



# write csv ###########
submission = open('submission.csv', 'w')    # get auch mit .csv bzw .txt
submission.write(str(number_of_total_commands) + '\n')
submission.writelines(submission_commands)
submission.close()
